In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA
from scipy.stats import chi2, norm
import random

from numpy.random import seed
import tensorflow as tf
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.layers import Dense 
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import regularizers
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import Adam

In [1]:
df = pd.read_csv("test_1.csv", parse_dates=["fecha_hora"])
pd.set_option('display.max_rows', None)  # Mostrar todas las filas
df.head()



NameError: name 'pd' is not defined

In [ ]:
# Seleccionar solo las columnas de Crest Factor (_FC)
fc_cols = [col for col in df.columns if col.endswith("_FC")]
df = df[["fecha_hora"] + fc_cols]  # Mantener fecha_hora para análisis temporal
df = df.set_index('fecha_hora')

# Revisar datos faltantes
df.isnull().sum()

In [ ]:
df.plot(figsize = (12,6))

In [ ]:
# Dividimos los datos train y test
train = df[:'2003-11-04 12:10:00']
test = df['2003-11-04 12:00:00':]

In [ ]:
#Se normalizan los datos unsando MinMaxScaler

scaler = MinMaxScaler()

X_train = pd.DataFrame(scaler.fit_transform(train), columns=train.columns, index=train.index)

X_test = pd.DataFrame(scaler.transform(test),  columns=test.columns, index=test.index)

In [ ]:
# Se hace una reduccion de dimensiones o dimensional para mejorar resultados usando PCA
# Se usa n_components=2 pero se usa segun la cantidad de varianza que se quiera conservar
pca = PCA(n_components=5, svd_solver= 'full')

# Se ajustan los datos 
X_train_PCA = pca.fit_transform(X_train)
X_train_PCA = pd.DataFrame(X_train_PCA)
X_train_PCA.index = X_train.index

X_test_PCA = pca.transform(X_test)
X_test_PCA = pd.DataFrame(X_test_PCA)
X_test_PCA.index = X_test.index

# Se puede ver los componentes principales 
pca.explained_variance_ratio_

In [ ]:
# Calcular varianza acumulada
var_acumulada = np.cumsum(pca.explained_variance_ratio_)

# Graficar
plt.plot(range(1, len(var_acumulada) + 1), var_acumulada, marker='o', linestyle='--')
plt.xlabel('Número de componentes')
plt.ylabel('Varianza acumulada')
plt.title('Varianza acumulada vs. Componentes principales')
plt.grid()
plt.show()

#### La idea es calcular la distancia de Mahalanobis con unos datos de entrenamiento que en este caso seria los datos de funcionamiento normal (los 2 primeros dias para este caso) establecer un umbral, para luego con los demas datos compararlos  poder establecer unas codiciones normales de funcionamiento de la maquina o dispositivo y ver en que momento el funcionamiento ya no es el optimo.

In [ ]:
# Estas son las funciones para calcular la distancia de Mahalanobis
def cov_matrix(data, verbose=False):
    covariance_matrix = np.cov(data, rowvar=False)
    if is_pos_def(covariance_matrix):
        inv_covariance_matrix = np.linalg.inv(covariance_matrix)
        if is_pos_def(inv_covariance_matrix):
            return covariance_matrix, inv_covariance_matrix
        else:
            print("Error: Inverse of Covariance Matrix is not positive definite!")
    else:
        print("Error: Covariance Matrix is not positive definite!") 


def MahalanobisDist(inv_cov_matrix, mean_distr, data, verbose=False):
    inv_covariance_matrix = inv_cov_matrix
    vars_mean = mean_distr
    diff = data - vars_mean
    md = []
    for i in range(len(diff)):
        md.append(np.sqrt(diff[i].dot(inv_covariance_matrix).dot(diff[i])))
    return md
    
    
def MD_detectOutliers(dist, extreme=False, verbose=False):
    k = 3. if extreme else 2.
    threshold = np.mean(dist) * k
    outliers = []
    for i in range(len(dist)):
        if dist[i] >= threshold:
            outliers.append(i)  # index of the outlier
    return np.array(outliers)


def MD_threshold(dist, extreme=False, verbose=False):
    k = 3. if extreme else 2.
    threshold = np.mean(dist) * k
    return threshold


def is_pos_def(A):
    if np.allclose(A, A.T):
        try:
            np.linalg.cholesky(A)
            return True
        except np.linalg.LinAlgError:
            return False
    else:
        return False

In [ ]:
# Se definen los datos de train y test usando los resultados obtenidos aplicando PCA
data_train = np.array(X_train_PCA.values)
data_test = np.array(X_test_PCA.values)

# Se calcula la matriz de covarianza y la inversa
cov_matrix, inv_cov_matrix  = cov_matrix(data_train)

# Se calcula la media de los datos de train
mean_distr = data_train.mean(axis=0)

# Se calcula la distancia de Mahalanobis para los datos de entrenamiento y se fija el umbral para marcar un punto de datos como anomalo
dist_train = MahalanobisDist(inv_cov_matrix, mean_distr, data_train, verbose=False)
threshold = MD_threshold(dist_train, extreme = True)

# Se calcula ahora la distancia para los datos de prueba para compararla con el umbral de anomalia dado 
dist_test = MahalanobisDist(inv_cov_matrix, mean_distr, data_test, verbose=False)

print('Calculated threshold value : ' + str(threshold))

#### Como se mencionó anteriormente, el cuadrado de la distancia de Mahalanobis al centroide de la distribución debería seguir una distribución χ² si se cumple el supuesto de que las variables de entrada se distribuyen normalmente. Este es también el supuesto en el que se basa el cálculo anterior del "valor umbral" para marcar una anomalía. Dado que es probable que este supuesto no se cumpla en este caso, conviene visualizar la distribución de la distancia de Mahalanobis para establecer un buen valor umbral para marcar anomalías.

In [ ]:
ax = sns.distplot(np.square(dist_train),
             bins = 20, 
             kde= False,
             fit=chi2,
             fit_kws={"label": "Chi squared fit"})
ax.set_xlim([0.0,15])
ax.set_ylim([0.0,0.4])
ax.legend()

In [ ]:
# Se grafica la distribucion de entrenamiento con un ajuste de una distribucion normal
# Suponiendo que dist_train es tu conjunto de datos
plt.figure()
sns.distplot(dist_train,
             bins = 10, 
             kde= True, 
            color = 'green');
plt.xlim([0.0,5])
plt.xlabel('Mahalanobis dist')

In [ ]:
# Se hace un dataframe con los datos de la distancia de mahalanubis de la data de prueba el umbral 
anomaly = pd.DataFrame()
anomaly['Mob dist']= dist_test
anomaly['Thresh'] = threshold

# Se compara los valores del umbral y de la distancia y se clasifica si es anomalia o no
anomaly['Anomaly'] = anomaly['Mob dist'] > anomaly['Thresh']
anomaly.index = X_test_PCA.index
anomaly.head()

In [ ]:
# Se hace un dataframe con los datos de la distancia de mahalanubis de la data de entrenamiento el umbral 
anomaly_train = pd.DataFrame()
anomaly_train['Mob dist']= dist_train
anomaly_train['Thresh'] = threshold

# Se compara los valores del umbral y de la distancia y se clasifica si es anomalia o no
anomaly_train['Anomaly'] = anomaly_train['Mob dist'] > anomaly_train['Thresh']
anomaly_train.index = X_train_PCA.index

# Se concatenan los datos de train y test
anomaly_alldata = pd.concat([anomaly_train, anomaly])
anomaly_train.head()

In [ ]:
# Se grafica para ver los resultados 
anomaly_alldata.plot(logy=True, 
                     figsize = (12,6), 
                     ylim = [1e-1,1e3], 
                     color = ['green','red'])

#### La idea básica es utilizar la red neuronal Autoencoder para comprimir las lecturas del sensor a una representación de baja dimensión, que captura las correlaciones e interacciones entre las distintas variables. (Esencialmente, el mismo principio que el modelo PCA, pero aquí también se consideran las no linealidades entre las características de entrada).

#### La red del autocodificador se entrena con datos que representan condiciones operativas normales, con el objetivo de comprimir y luego reconstruir las variables de entrada. Durante la reducción de dimensionalidad, la red aprende las interacciones entre las distintas variables y debería ser capaz de reconstruirlas a las variables originales en la salida. La idea es que, a medida que el equipo monitoreado se degrada o se producen anomalías, esto afecte la interacción entre las variables de entrada (por ejemplo, cambios de temperatura, presión, etc.). En este caso, se observará un aumento del error en la reconstrucción de las variables de entrada por parte de la red. Al monitorear el error de reconstrucción, se puede obtener una indicación del estado del equipo monitoreado, ya que este error aumentará a medida que este se degrada.

In [ ]:
seed(10)
np.random.seed(10)
tf.random.set_seed(10)
act_func = 'elu'
# Input layer:
model=Sequential()
# First hidden layer, connected to input vector X. 
model.add(Dense(10,activation=act_func,
                kernel_initializer='glorot_uniform',
                kernel_regularizer=regularizers.l2(0.001),
                input_shape=(X_train.shape[1],)
               )
         )
model.add(Dense(2,activation=act_func,
                kernel_initializer='glorot_uniform'))

model.add(Dense(10,activation=act_func,
                kernel_initializer='glorot_uniform'))

model.add(Dense(X_train.shape[1],
                kernel_initializer='glorot_uniform'))

model.compile(loss='mse',optimizer='adam')

# Train model for 100 epochs, batch size of 10
NUM_EPOCHS=50
BATCH_SIZE=64

model.summary()

In [ ]:
history=model.fit(np.array(X_train),np.array(X_train),
                  batch_size=BATCH_SIZE, 
                  epochs=NUM_EPOCHS,
                  validation_split=0.05,
                  verbose = 1)

In [ ]:
plt.plot(history.history['loss'],
         'b',
         label='Training loss')
plt.plot(history.history['val_loss'],
         'r',
         label='Validation loss')
plt.legend(loc='upper right')
plt.xlabel('Epochs')
plt.ylabel('Loss, [mse]')
plt.ylim([0,.1])
plt.show()

In [ ]:
X_pred = model.predict(np.array(X_train))
X_pred = pd.DataFrame(X_pred, 
                      columns=X_train.columns)
X_pred.index = X_train.index

scored = pd.DataFrame(index=X_train.index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_train), axis = 1)

In [ ]:
plt.figure()
sns.distplot(scored['Loss_mae'],
            bins=10,
            kde=True,
            color='blue');
plt.title("Loss distribution ,training set")
plt.xlim([0.0, .5])  # set y-axis range

In [ ]:
Q1 = scored['Loss_mae'].quantile(0.25)
Q3 = scored['Loss_mae'].quantile(0.75)
RIC = Q3 - Q1
LS = Q3 + 2 * RIC
LS

In [ ]:
threshold = np.mean(scored['Loss_mae']) + 4*np.std(scored['Loss_mae'])
threshold

In [ ]:
thresh = 0.25

In [ ]:
X_pred = model.predict(np.array(X_test))
X_pred = pd.DataFrame(X_pred, 
                      columns=X_test.columns)
X_pred.index = X_test.index

scored = pd.DataFrame(index=X_test.index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_test), axis = 1)
scored['Threshold'] = thresh
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']
scored.head()

In [ ]:
X_pred_train = model.predict(np.array(X_train))
X_pred_train = pd.DataFrame(X_pred_train, 
                      columns=X_train.columns)
X_pred_train.index = X_train.index

scored_train = pd.DataFrame(index=X_train.index)
scored_train['Loss_mae'] = np.mean(np.abs(X_pred_train-X_train), axis = 1)
scored_train['Threshold'] = thresh
scored_train['Anomaly'] = scored_train['Loss_mae'] > scored_train['Threshold']

In [ ]:
scored = pd.concat([scored_train, scored])
scored.plot(logy=True,  figsize = (10,6), ylim = [1e-2,1e2], color = ['blue','red'])